**IMAGE CLASSIFICATION (CATS VS DOGS) WITH TRANSFER LEARNING (VGG16 ALGORITHM)**


In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import os
import tensorflow as tf

In [4]:
base_dir= r"/content/drive/MyDrive/Dataset/dog_vs_cat/dataset"

In [5]:
#preprocessing
image_sz=224
batch_sz=16

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(

    
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1)

validation_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    
    validation_split=0.1
)

In [6]:
#train_Data_Portion
train_generator=train_datagen.flow_from_directory(
    base_dir,
    target_size=(image_sz,image_sz),
    batch_size=batch_sz,
    subset='training'

)

validation_generator=validation_datagen.flow_from_directory(
    base_dir,
    target_size=(image_sz,image_sz),
    batch_size=batch_sz,
    subset='validation'

)

Found 1800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [7]:
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from glob import glob

In [8]:
#in vgg16, image size must be 224*224
image_size=[224,224]
vgg=VGG16(input_shape=image_size+[3], weights='imagenet', include_top=False)
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [9]:
#we will not going to train vgg layers
for layer in vgg.layers:
    layer.trainable=False

In [12]:
#to make the vgg16 recognise our dataclass from folders/directory
folders=glob(r"/content/drive/MyDrive/Dataset/dog_vs_cat/dataset/*")
print(len(folders))

2


In [13]:
x=Flatten()(vgg.output)
prediction=Dense(len(folders),activation='softmax')(x) #we can add more dense layer here to increase acuracy
model=Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [14]:
#model_train
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
epoch=3

history= model.fit(train_generator,
                  steps_per_epoch=len(train_generator),
                  epochs=epoch,
                  validation_data=validation_generator,
                  validation_steps=len(validation_generator)
                 )

Epoch 1/3
113/113 [==============================] - 669s 6s/step - loss: 0.1671 - accuracy: 0.9306 - val_loss: 0.2594 - val_accuracy: 0.8950
Epoch 2/3
113/113 [==============================] - 669s 6s/step - loss: 0.1328 - accuracy: 0.9522 - val_loss: 0.2121 - val_accuracy: 0.9100
Epoch 3/3
113/113 [==============================] - 672s 6s/step - loss: 0.1439 - accuracy: 0.9417 - val_loss: 0.3551 - val_accuracy: 0.8750


In [29]:
 # Model Prediction

from keras.preprocessing import image
import numpy as np

img_pred=image.load_img(r"/content/drive/MyDrive/Dataset/dog_vs_cat/dataset/cats/cat.4027.jpg",target_size=(224,224))

img_pred=image.img_to_array(img_pred)
img_pred=np.expand_dims(img_pred, axis=0)


rslt = model.predict(img_pred)

print(rslt)
if rslt[0][0]>rslt[0][1]:
    prediction="cat"
    
    
else:
    prediction="dog"
print(prediction)


[[1. 0.]]
cat
